In [2]:
import tensorflow as tf
from tensorflow.keras import models, Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
import os

In [3]:
base_dir='intel_dataset'
train_dir= os.path.join(base_dir,'seg_train/seg_train')
test_dir= os .path.join(base_dir,'seg_test/seg_test')

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)
test_datagen=ImageDataGenerator(rescale=1./255)

In [8]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 11230 images belonging to 6 classes.


In [9]:
validation_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2804 images belonging to 6 classes.


In [10]:
test_generator=train_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

Found 3000 images belonging to 6 classes.


In [11]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(6, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [13]:
optimizer=Adam(learning_rate=0.0002)

In [14]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 20
)

Epoch 1/20
100/100 [==============================] - 38s 372ms/step - loss: 1.3237 - accuracy: 0.4753 - val_loss: 1.1305 - val_accuracy: 0.5391
Epoch 2/20
100/100 [==============================] - 32s 316ms/step - loss: 1.0436 - accuracy: 0.5941 - val_loss: 1.0574 - val_accuracy: 0.5688
Epoch 3/20
100/100 [==============================] - 30s 295ms/step - loss: 0.9525 - accuracy: 0.6351 - val_loss: 0.9000 - val_accuracy: 0.6469
Epoch 4/20
100/100 [==============================] - 29s 292ms/step - loss: 0.8765 - accuracy: 0.6575 - val_loss: 0.8015 - val_accuracy: 0.6891
Epoch 5/20
100/100 [==============================] - 28s 284ms/step - loss: 0.8161 - accuracy: 0.6941 - val_loss: 0.7873 - val_accuracy: 0.7031
Epoch 6/20
100/100 [==============================] - 29s 285ms/step - loss: 0.7657 - accuracy: 0.7067 - val_loss: 0.6840 - val_accuracy: 0.7422
Epoch 7/20
100/100 [==============================] - 28s 275ms/step - loss: 0.7277 - accuracy: 0.7319 - val_loss: 0.6753 - val_ac

In [16]:
_, accuracy = model.evaluate(test_generator)

print(f'accuracy of the model : {accuracy}')

94/94 [==============================] - 16s 174ms/step - loss: 0.5397 - accuracy: 0.8077
accuracy of the model : 0.8076666593551636


In [17]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(
    weights='imagenet',
    include_top = True,
    input_shape = (224, 224, 3)
)

In [18]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
img_path = 'intel_dataset/seg_test/seg_test/buildings/20057.jpg'

In [20]:
from tensorflow.keras.preprocessing import image as keras_image

img_test = keras_image.load_img(img_path, target_size=(224, 224))
img_test = keras_image.img_to_array(img_test)

In [21]:
img_test.shape

(224, 224, 3)

In [23]:
prediction_test = base_model.predict(img_test.reshape(1, 224, 224, 3))

1/1 [==============================] - 0s 218ms/step


In [24]:
prediction_test.argmax()

437

In [26]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape = (150, 150, 3)
)

58889256/58889256 [==============================] - 26s 0us/step


In [27]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [28]:
for layer in base_model.layers:
    layer.trainable = False

In [29]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model

output_vgg = base_model.output
global_avg_pooling = GlobalAveragePooling2D() (output_vgg)

dense_1 = Dense(512, activation='relu', name='dense_1') (global_avg_pooling)
# dropout_1 = Dropout(0.2, name='dropout_1') (dense_1)
dense_2 = Dense(256, activation='relu', name='dense_2') (global_avg_pooling)
dropout_2 = Dropout(0.2, name='dropout_2') (dense_2)
output = Dense(6, activation='softmax', name='output') (dropout_2)

transfer_model = Model(inputs=base_model.input , outputs= output)

In [30]:
transfer_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [31]:
# Compiler le modèle
optimizer = Adam(learning_rate=0.002)


transfer_model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
transfer_model.fit(
    train_generator,
    steps_per_epoch = 200,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 20
)

Epoch 1/20
200/200 [==============================] - 125s 620ms/step - loss: 0.6125 - accuracy: 0.7669 - val_loss: 0.5048 - val_accuracy: 0.8062
Epoch 2/20
200/200 [==============================] - 124s 618ms/step - loss: 0.4354 - accuracy: 0.8343 - val_loss: 0.3781 - val_accuracy: 0.8687
Epoch 3/20
200/200 [==============================] - 126s 628ms/step - loss: 0.4014 - accuracy: 0.8512 - val_loss: 0.3696 - val_accuracy: 0.8484
Epoch 4/20
200/200 [==============================] - 125s 623ms/step - loss: 0.3683 - accuracy: 0.8606 - val_loss: 0.3545 - val_accuracy: 0.8609
Epoch 5/20
200/200 [==============================] - 123s 616ms/step - loss: 0.3637 - accuracy: 0.8648 - val_loss: 0.3659 - val_accuracy: 0.8656
Epoch 6/20
200/200 [==============================] - 123s 617ms/step - loss: 0.3421 - accuracy: 0.8720 - val_loss: 0.3998 - val_accuracy: 0.8422
Epoch 7/20
200/200 [==============================] - 123s 617ms/step - loss: 0.3295 - accuracy: 0.8795 - val_loss: 0.3552 -